# Developing Activity class
__Keith Cheveralls__<br>
__March 2019__



In [ ]:
import os
import re
import sys
import gzip
import time
import pickle
import datetime
import numpy as np
from scipy import stats
import pandas as pd

import fitparse
from fitparse import FitFile
from matplotlib import pyplot as plt

%matplotlib 
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.append('../')
import cypy2

root = '/home/keith/Downloads/export_7989839-1'

wahoo_example = '2326365683.fit.gz'
garmin_example = '2122584483.fit.gz'
garmin_indoor_example = '2324139976.fit.gz'

In [ ]:
cypy2.ActivityManager

### Load a single activity directly from a FIT file

In [ ]:
a = cypy2.Activity.from_fit(os.path.join(root, 'activities', garmin_example))

In [ ]:
a.metadata

### Load all activities from cached strava export

In [ ]:
strava_export = cypy2.StravaExportManager(root, from_cache=True)

In [ ]:
m = cypy2.ActivityManager.from_strava_export(strava_export)

In [ ]:
cypy2.Activity.id_from_fit(file_id=pd.Series(dict(time_created=pd.to_datetime('2018-01-02 3:45:56'))))

In [ ]:
cypy2.Activity.id_from_fit(file_id=pd.Series(dict(time_created=pd.to_datetime('2018-01-02'))))

In [ ]:
# check activity types
m.metadata().activity_type.unique()

In [ ]:
# count sensor flags by device
md = m.metadata()
pd.concat((md.groupby('device_model').count().activity_id, md.groupby('device_model').sum()), axis=1)

In [ ]:
# rides with power but without heart rate
m.metadata(activity_type='ride', heart_rate_flag=False, power_flag=True)

In [ ]:
# rides with heart rate but without an antplus_device_type column in device_info
# these *should* be only runs from fenix3
ids_wo_antplus = []
for a in m.activities(activity_type='run'):
    if 'antplus_device_type' not in a._fit_data['device_info'].columns:
        if 'heart_rate' in a._fit_data['record'].columns:
            ids_wo_antplus.append(a.metadata['activity_id'])
            
len(ids_wo_antplus)

In [ ]:
# rides with cadence but no power - should all be from fr220
ids_w_cad_wo_power = []
for a in m.activities(activity_type='ride'):
    columns = a._fit_data['record'].columns
    if ('cadence' in columns) and ('power' not in columns):
        ids_w_cad_wo_power.append(a.metadata['activity_id'])
        print(a.metadata['device_model'])
len(ids_w_cad_wo_power)

In [ ]:
m.activities()[-1]._fit_data['event']

In [ ]:
# cat all messages for some activities
dcat = pd.concat(tuple([a._fit_data['device_info'] for a in m.activities(device_model='edge520')]))

In [ ]:
dcat